<a href="https://colab.research.google.com/github/happyhillll/DigitalHumanities/blob/main/%5BDH%EA%B2%A8%EC%9A%B8%ED%95%99%EA%B5%90%5D_JaneAusten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제인 오스틴 대화 데이터 분석 (project-dialogism-novel-corpus)
작성 : 김병준(KAIST 디지털인문사회과학센터) / 정서현(KAIST 디지털인문사회과학부)  
주의 : 2023 디지털인문학 겨울학교 문학 트랙 day3 코딩 실습 자료입니다. 아직 진행중인 연구이므로 활용에 주의해주십시오.

### 0. 필요 패키지 설치 및 로드
상단 메뉴에서 런타임 - 런타임 유형 변경을 눌러 GPU임을 확인

In [ ]:
pip install -q pca gensim nltk 'spacy[cuda-autodetect]' transformers gdown natsort gutenbergpy

In [ ]:
# spacy 영어 모델 다운로드(다운로드 속도를 위해 sm(small) 모델 다운)
!python -m spacy download en_core_web_sm

In [ ]:
# 필요 패키지 load
import spacy
print(spacy.prefer_gpu()) #GPU 활용
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.add_pipe('sentencizer')
import pandas as pd
from pca import pca
from tqdm import tqdm
tqdm.pandas()
import ast
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from collections import Counter
from itertools import chain

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.utils import simple_preprocess
import gensim
import numpy as np
from pprint import pprint

from transformers import pipeline
ekman = pipeline('sentiment-analysis', model='arpanghoshal/EkmanClassifier', device=0, top_k=None)

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200 # DPI 고화질로 향상
import seaborn as sns
import gdown
from natsort import natsorted, index_natsorted, order_by_index
import gutenbergpy.textget

In [ ]:
# PDNC 데이터 github에서 clone
# https://github.com/Priya22/project-dialogism-novel-corpus
!git clone https://github.com/Priya22/project-dialogism-novel-corpus.git

### 1. 전처리

##### 제인 오스틴 소설 quote 자료 로드
* Emma  
* NorthangerAbbey  
* Persuasion  
* PrideAndPrejudice  
* SenseAndSensibility  

In [ ]:
# 제인 오스틴 5개 작품(폴더명) 리스트로 만들기
folder_list = ['Emma', 'NorthangerAbbey','Persuasion','PrideAndPrejudice','SenseAndSensibility']

In [ ]:
# df라는 변수에 제인 오스틴 대화 데이터 넣기
df = pd.DataFrame()
for folder in tqdm(folder_list):
    temp = pd.read_csv(f'./project-dialogism-novel-corpus/data/{folder}/quotation_info.csv')
    temp['title'] = folder # title 컬럼 추가
    df = pd.concat([df,temp],ignore_index=True)

In [ ]:
# 대화 데이터 확인
df

In [ ]:
# 작품별 대화(quote) 수
df['title'].value_counts()

In [ ]:
# subQuotationList : str to list 
df['subQuotationList'] = df['subQuotationList'].map(lambda x:ast.literal_eval(x))
df['subQuotationList']

In [ ]:
# quoteText와 subQuotationList의 차이 비교
pprint(df['quoteText'][5273])
pprint(df['subQuotationList'][5273])

In [ ]:
# quoteText를 문장 단위로 분절화 (spacy 모듈 활용)
df['sents'] = df['quoteText'].progress_map(lambda x:list(nlp(x).sents))

In [ ]:
# 문장 분절화 결과 확인
df['sents'][0]

In [ ]:
# 문장 단위로 행 확장 (5278개 행에서 14854행으로 확장)
df = df.explode('sents').reset_index(drop=True)
df

In [ ]:
# 캐릭터별 발화 문장 수
df.groupby(['title'])['speaker'].value_counts().to_excel('./작품별_인물_발화문장수.xlsx') # 엑셀로 저장
df.groupby(['title'])['speaker'].value_counts()

In [ ]:
# 작품-인물별 발화 토큰 수
df['numOfTokens'] = df['sents'].str.len()
df.groupby(['title','speaker'])['numOfTokens'].sum().to_excel('./작품별_인물_발화토큰수.xlsx')
df.groupby(['title','speaker'])['numOfTokens'].sum()

In [ ]:
# 작품-인물별 발화 글자수(띄어쓰기 포함)
df['numOfLetter'] = df['quoteText'].str.len()
df.groupby(['title','speaker'])['numOfLetters'].sum().to_excel('./작품별_인물_발화글자수.xlsx')
df.groupby(['title','speaker'])['numOfLetters'].sum()

In [ ]:
df.groupby(['title','speaker']).agg({'speaker':lambda x:x.value_counts(),
                                    'numOfTokens':'sum',
                                    'numOfLetters':'sum'}).to_excel('./작품별_인물_발화문장토큰글자수.xlsx')

df.groupby(['title','speaker']).agg({'speaker':lambda x:x.value_counts(),
                                    'numOfTokens':'sum',
                                    'numOfLetters':'sum'})

### 2. 토크나이징
[유명한 자연어처리 패키지 spaCy](https://spacy.io/)  
[품사 tag 참고](https://universaldependencies.org/u/pos/)

In [ ]:
# 토크나이징 
# https://spacy.io/usage/linguistic-features
df['tokens'] = df['sents'].progress_map(lambda x:[token.lemma_+'/'+token.pos_ for token in x]) #Lemmatization 처리된 토큰 추출 (https://wikidocs.net/21707)
df['tokens']

In [ ]:
# 토크나이징 결과 확인
df['tokens'][0]

In [ ]:
# 상위 n개 Unigram 확인
unigram = chain(*df['tokens'].tolist())
cnt = Counter(unigram)
cnt.most_common(50) # 상위 N개

##### 특정 품사만 추출하기

In [ ]:
allowed_postags = ['ADJ','NOUN','VERB','PROPN','ADV'] # 추출하고 싶은 품사 리스트 (형용사, 명사, 동사, 고유명사, 부사)

In [ ]:
df['allowed_tokens'] = df['tokens'].map(lambda x:[token for token in x if token.split('/')[1] in allowed_postags])
df['allowed_tokens'] 

In [ ]:
def cal_unigram(tokens):
    unigram = chain(*tokens)
    cnt = Counter(unigram)
    res = cnt.most_common(50) # 상위 N개
    return res

In [ ]:
# 작품 - 캐릭터별 상위 50개 토큰 정리 엑셀로 저장
for idx, title in tqdm(enumerate(folder_list)):
    df_title = df.loc[df['title']==title]
    chas = list(df_title['speaker'].unique())
    title_freq = pd.DataFrame()
    for cha in chas:
        title_freq = pd.concat([title_freq,pd.DataFrame(cal_unigram(df_title.loc[df_title['speaker']==cha,'allowed_tokens'].tolist()),columns=[f'{cha}','freq'])], axis=1)
    if idx==0:
        title_freq.to_excel('./title_allowed_tokens_freq.xlsx',sheet_name=title,index=None)
    else:
        with pd.ExcelWriter(f'./title_allowed_tokens_freq.xlsx',mode='a',engine='openpyxl') as writer:
            title_freq.to_excel(writer,sheet_name=title,index=None)

In [ ]:
# 예시
title_freq

##### 불용어(stopwords) 처리
보통 텍스트 마이닝 연구에서 별 의미가 없거나 문법적인 역할만 하는 단어들을 제거하는 것.

In [ ]:
# nltk 불용어의 문제 (https://www.nltk.org/), 과연 문학 텍스트에 적용해도 되는가?
stop_words = stopwords.words('english')
stop_words

In [ ]:
# 상위 n개 Unigram 확인
unigram = chain(*df['allowed_tokens'])
cnt = Counter(unigram)
cnt.most_common(100) # 상위 N개

In [ ]:
stop_words = ['Mr./PROPN','Mrs./PROPN','Miss/PROPN']

In [ ]:
# 불용어 제거
df['allowed_tokens_stop'] = df['allowed_tokens'].map(lambda x:[t for t in x if not t in stop_words])
df['allowed_tokens_stop']

In [ ]:
# 상위 n개 Unigram 확인
unigram = chain(*df['allowed_tokens_stop'])
cnt = Counter(unigram)
cnt.most_common(100) # 상위 N개

### 3. 감정 분류(Emotion Classification)
https://en.wikipedia.org/wiki/Emotion_classification  
[GoEmotions](https://ai.googleblog.com/2021/10/goemotions-dataset-for-fine-grained.html)  
[GoEmotions를 개량화한 모델 활용](https://huggingface.co/arpanghoshal/EkmanClassifier)

In [ ]:
# GoEmotions에서 ekman(7개 감정으로 경량화) 모델 연습
ekman('I cannot agree with you, papa; you know I cannot.') # 영어 문장을 넣어보세요(실습)

In [ ]:
# 각 문장의 감정 점수 추출 (3~4분 소요)
df['emotions'] = df['sents'].progress_map(lambda x:ekman(str(x)))

In [ ]:
# 결과 확인
pprint(df['sents'][0])
pprint(df['emotions'][0])

In [ ]:
# 감정 알파벳순으로 정렬 (현재 가장 점수 높은 감정이 맨앞에 있음)
df['emotions'] = df['emotions'].map(lambda x:sorted(x[0], key=lambda d: d['label']))
df['emotions']

In [ ]:
#  감정 테이블 생성(df_emo)
df['emotions_scores'] = df['emotions'].map(lambda x:[s['score'] for s in x])
df_emo = pd.DataFrame()
for emos in tqdm(df['emotions_scores'].tolist()):
    temp = pd.DataFrame.from_dict(emos).T
    df_emo = pd.concat([df_emo,temp],ignore_index=True)
df_emo.columns = ['anger','disgust','fear','joy','neutral','sadness','surprise']
df_emo

In [ ]:
# Jane Austen 소설의 7개 감정 통계량
df_emo.describe()

In [ ]:
# Jane Austen 소설의 7개 박스 플롯
# 박스플롯 이해하기 : https://newsjel.ly/archives/newsjelly/14177
sns.boxplot(data = df_emo)

In [ ]:
# 문장-감정 테이블(df_emo)에 화자와 quoteID, 작품명 추가 
df_emo['speaker'] = df['speaker']
df_emo['quoteID'] = df['quoteID']
df_emo['title'] = df['title']
df_emo.to_excel('./df_emo.xlsx')
df_emo

##### 작품/캐릭터별 감정 통계 및 박스플롯 시각화

In [ ]:
# 5개 소설의 주요 캐릭터 7명(총 35명)을 항목별로 정리한 파일 다운로드(feat 정서현 교수님)
url = 'https://docs.google.com/uc?id=12nQcV9dugVGo5LS7hoAvF0odO_sJXpg0'
file = 'JaneAusten_Characters.xlsx'
gdown.download(url,file)

In [ ]:
characters_df = pd.read_excel('./JaneAusten_Characters.xlsx')
characters_df

In [ ]:
# 각 작품별로 캐릭터 리스트 생성
SenseAndSensibility = characters_df.iloc[0,1:].tolist()
NorthangerAbbey = characters_df.iloc[1,1:].tolist()
PrideAndPrejudice = characters_df.iloc[2,1:].tolist()
Emma = characters_df.iloc[3,1:].tolist()
Persuasion = characters_df.iloc[4,1:].tolist()
characters = list(characters_df.iloc[:,1:].stack().values) # 전체 35명의 캐릭터
characters

In [ ]:
# 작품-캐릭터 딕셔너리 생성
title_chas = [{'SenseAndSensibility':SenseAndSensibility},{'NorthangerAbbey':NorthangerAbbey},{'PrideAndPrejudice':PrideAndPrejudice},{'Emma':Emma},{'Persuasion':Persuasion}]
title_chas

In [ ]:
# 작품별로 주요 7명의 캐릭터들의 감정 통계량과 박스플롯 시각화 파일 저장(plots 폴더아래)
!mkdir plots # plots 폴더 생성
emotions =  ['anger', 'disgust','fear', 'joy','neutral','sadness','surprise']
for title_dict in tqdm(title_chas):
    chas = list(title_dict.values())[0]
    title = list(title_dict.keys())[0]
    temp_df = df_emo.loc[df_emo['speaker'].isin(chas)]
    # 통계량
    temp_df.describe().to_excel(f'./plots/{title}_describe.xlsx')
    # 감정별 박스플롯
    for emo in emotions:
        emo_boxplot = sns.boxplot(x='speaker',y=emo, data=temp_df)
        emo_boxplot.set_xticklabels(emo_boxplot.get_xticklabels(), rotation=45)
        emo_boxplot_fig = emo_boxplot.get_figure()
        emo_boxplot_fig.savefig(f'./plots/{title}_{emo}_boxplot.png', dpi=300, bbox_inches='tight')
        emo_boxplot_fig.clf()

In [ ]:
# 작품별 박스플롯
for emo in ['anger', 'disgust','fear', 'joy','neutral','sadness','surprise']:
    emo_boxplot = sns.boxplot(x='title',y=emo, data=df_emo)
    emo_boxplot.set_xticklabels(emo_boxplot.get_xticklabels(), rotation=45)
    emo_boxplot_fig = emo_boxplot.get_figure()
    emo_boxplot_fig.savefig(f'./plots/title_{emo}_boxplot.png', dpi=300, bbox_inches='tight')
    emo_boxplot_fig.clf()

##### 대화 진행에 따른 시계열 감정 추이

In [ ]:
df_emo['quoteNum'] = df_emo['quoteID'].str.extract('(\d+)').astype(int)
df_emo_quote = df_emo.groupby(['title','quoteNum','speaker']).mean()
df_emo_quote

In [ ]:
# 작품 Emma의 quoteID 진행에 따른 감정 추이 
Emma_emo = df_emo_quote.xs('Emma',level='title').reset_index()
Emma_emo = Emma_emo.reset_index()
Emma_emo['index'] = (Emma_emo['index'] + 1) / len(Emma_emo)
Emma_emo

In [ ]:
# 기쁨의 추세선
sns.regplot(x='index',y='joy',data=Emma_emo)

In [ ]:
# line plot
Emma_emo.groupby('index').mean()['joy'].plot()

In [ ]:
# quote 단위가 너무 작기 때문에 감정 변화의 추이를 살펴보기 어려움. 따라서 좀더 큰 범위의 대화 범위를 추가 마킹(feat. 정서현 교수님)
# 두개 작품(이성과 감성, 오만과 편견만 추가 marking)
url = 'https://docs.google.com/uc?id=1KeoJ3PP5rFcBVdQn9zJBO6u-T2ugOMOt'
file = 'SenseAndSensibilityConversationsMarked.xlsx'
gdown.download(url,file)

url = 'https://docs.google.com/uc?id=1CyviGAW9Ajvt-ONZowyet97emv8AMHWb'
file = 'PrideAndPrejudiceConversationsMarked.xlsx'
gdown.download(url,file)

In [ ]:
# 이성과 감성
SASCM = pd.read_excel('./SenseAndSensibilityConversationsMarked.xlsx')
SASCM #conversationID 추가함

In [ ]:
df_emo_SS = df_emo.loc[df_emo['title']=='SenseAndSensibility'].reset_index(drop=True)
df_emo_SS = df_emo_SS.merge(SASCM[['quoteID','conversationID']],on='quoteID',how='inner')
df_emo_SS

In [ ]:
# 이성과 감성 감정 추이
df_emo_SS_cID = df_emo_SS.groupby(['conversationID']).mean().iloc[:,:-1] # 평균 활용
df_emo_SS_cID.index = natsorted(df_emo_SS_cID.index)
df_emo_SS_cID

In [ ]:
df_emo_SS_cID.plot()

In [ ]:
# 엑셀로 저장
df_emo_SS_cID.to_excel('SenseAndSensibility_emo_cID.xlsx')

In [ ]:
# 오만과 편견
PAPCM = pd.read_excel('./PrideAndPrejudiceConversationsMarked.xlsx')
PAPCM

In [ ]:
df_emo_PP = df_emo.loc[df_emo['title']=='PrideAndPrejudice'].reset_index(drop=True)
df_emo_PP = df_emo_PP.merge(PAPCM[['quoteID','conversationID']],on='quoteID',how='inner')

In [ ]:
# 오만과 편견 감정 추이
df_emo_PP_cID = df_emo_PP.groupby(['conversationID']).mean().iloc[:,:-1] # 평균 활용
df_emo_PP_cID.index = natsorted(df_emo_PP_cID.index)
df_emo_PP_cID

In [ ]:
df_emo_PP_cID.plot()

In [ ]:
# 엑셀로 저장
df_emo_PP_cID.to_excel('PrideAndPrejudice_emo_cID.xlsx')

### 4. PCA(주성분분석)
7개의 감정을 2개의 축으로 축소하여 어떤 감정이 중요한 변수인지 확인하는 방법  
참고문헌 : [김병준, 전봉관 and 이원재. (2017). 비평 언어의 변동: 문예지 비평 텍스트에 나타난 개념단어의 변동 양상, 1995~2015. 현대문학의 연구, 61](https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART002201115)

In [ ]:
# 주요 35명의 캐릭터만 활용
df_emo_main = df_emo[df_emo['speaker'].isin(characters)].reset_index(drop=True).iloc[:,:-1]
df_emo_main

In [ ]:
# 주요 캐릭터들의 감정 평균
char_emo = df_emo_main.groupby(['speaker']).mean()
# char_emo = char_emo.reset_index()
char_emo

In [ ]:
# Reduce the data towards 2 PCs
sents_cha_model = pca(n_components=2)

In [ ]:
# Fit transform
X = df_emo_main.iloc[:,:7]
y = df_emo_main.iloc[:,9]
labels = df_emo_main.iloc[:,:7].columns.tolist()
# results = model.fit_transform(X,col_labels=char_emo.iloc[:,1:].columns,row_labels=y)
sents_cha_results = sents_cha_model.fit_transform(X, col_labels=labels, row_labels=y)

In [ ]:
# Cumulative explained variance
print(sents_cha_model.results['explained_var'])

In [ ]:
# Explained variance per PC
print(sents_cha_model.results['variance_ratio'])

In [ ]:
# 2D plot
fig, ax = sents_cha_model.scatter(label=None)

In [ ]:
sents_cha_model.biplot(cmap=None, label=None, legend=False)

In [ ]:
sents_cha_model.results['topfeat']

In [ ]:
# 캐릭터들의 감정 평균 활용 PCA
cha_model = pca(n_components=2)
cha_results = cha_model.fit_transform(char_emo)

In [ ]:
# biplot
cha_model.biplot(cmap=None, label=None, legend=False)

In [ ]:
# 인물들의 감정 평균값을 기반으로한 지형도
cha_model.scatter()

In [ ]:
# outlier
cha_model.scatter(SPE=True)

### 5. 네트워크 Edge list 만들기 (Gephi, networkX에서 활용 가능)

In [ ]:
df_net = pd.DataFrame()
for folder in tqdm(folder_list):
    temp = pd.read_csv(f'./project-dialogism-novel-corpus/data/{folder}/quotation_info.csv')
    temp['title'] = folder
    df_net = pd.concat([df_net,temp],ignore_index=True)

##### 청자-화자 Edge list 생성

In [ ]:
# addressees : str to list 
df_net['addressees'] = df_net['addressees'].map(lambda x:ast.literal_eval(x))

In [ ]:
# edge weight 는 1 / 청자수
# A화자에 B,C 청자가 2명이라면 해당 edge list의 weight는 각 0.5
df_net['weight'] = 1 / df_net['addressees'].str.len()
df_net['weight']

In [ ]:
df_net = df_net.explode('addressees').reset_index(drop=True)
df_net

In [ ]:
# 확인
df_net[['title','quoteID','speaker','addressees','weight']]

##### 작품별 edge list 엑셀로 저장
* network 폴더에 저장

In [ ]:
!mkdir network
for folder in tqdm(folder_list):
    Edge = df_net.loc[df_net['title']==folder,['quoteID','speaker','addressees','weight']].reset_index(drop=True)
    Edge.columns = ['quoteID','source','target','weight']
    Edge.to_excel(f'./network/{folder}_edge.xlsx',index=None)

In [ ]:
# 확인
Edge

### 6. 구텐베르크 프로젝트 가져오기
[Project Gutenberg](https://www.gutenberg.org/)  
[파이썬에서 구텐베르크 프로젝트 자료 자동 수집(gutenbergpy)](https://github.com/raduangelescu/gutenbergpy)

In [ ]:
def usage_example(id):
    # This gets a book by its gutenberg id number
    raw_book = gutenbergpy.textget.get_text_by_id(id) # with headers
    clean_book = gutenbergpy.textget.strip_headers(raw_book) # without headers
    return clean_book, raw_book

In [ ]:
# 필경사 바틀비(11231)
# https://www.gutenberg.org/ebooks/11231
cleaned_book, raw_book = usage_example(11231)

In [ ]:
# 각종 메타정보 포함한 버전(''The Project Gutenberg eBook of Bartleby, The Scrivener, by Herman Melvil ...')
pprint(raw_book)

In [ ]:
# 각종 메타 정보 삭제한 버전
pprint(cleaned_book)